# Setup

In [ ]:
source("/home/vickydiliscia/dmeyf2025/src/churn/5002/config.r")

In [ ]:
# Cargo el index (que a su vez definirá la función)
source(file.path(PATH_UTILS, "index.R"))

# Ejecuto la función que carga todos los utils
source_utils(PATH_UTILS)

## Parametros Experimento

In [ ]:
## Semillas
PARAM <- list()
PARAM$semilla_primigenia <- SEEDS[1]

# training y future
PARAM$train <- c(202102)
PARAM$train_final <- c(202101,202102,202103,202104)
PARAM$future <- c(202104)
PARAM$future_final <- c(202106)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

# undersampling
PARAM$trainingstrategy$undersampling <- 0.1

# carpeta de trabajo
setwd(PATH_THIS_EXPERIMENT)

## Parametros LGBM

In [3]:
PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)

ERROR: Error: object 'PARAM' not found


## librerias

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

## Dataset

In [ ]:
# lectura del dataset
dataset <- fread(DATASET_FEATURES, stringsAsFactors= TRUE)

In [ ]:
# clase01 - me daba error con los max_bin
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

# Hiperparametros

In [ ]:
setwd(PATH_BO)

In [ ]:
kbayesiana <- "bayesiana.RDATA"  # tu archivo con el opt.state adentro
# Cargar el archivo (esto mete en memoria los objetos guardados, p. ej. opt.state)
load(kbayesiana)

# Si el archivo contiene el estado de la optimización:
bayesiana_salida <- list(opt.path = opt.state$opt.path)
                         
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

PARAM$out$lgbm$y <- tb_bayesiana[1, y]

write_yaml( PARAM, file="PARAM.yml")

print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

## Ensamble

In [ ]:
setwd(PATH_MODELS)

In [ ]:
model_paths <- list.files(pattern = paste0("^modelo_", TAG, "_seed.*\\.txt$"))
modelos <- lapply(model_paths, lgb.load)

In [ ]:
setwd(PATH_THIS_EXPERIMENT)
prediction <- "prediction"
dir.create(prediction, showWarnings= FALSE)
setwd(PATH_PREDICTION)

In [ ]:
ensamblar_promedio (
  dataset       = dataset,
  PARAM         = PARAM,
  campos_buenos = campos_buenos,
  experimento   = TAG,
  models_dir    = PATH_MODELS,
  pattern       = paste0("^modelo_", TAG, "_seed.*\\.txt$")
)


# Envios a Kaggle

In [ ]:
# Leer archvio predicciones
setwd(PATH_PREDICTION)
pred_file <- paste0("prediccion_",TAG, ".txt")

# Verificá que exista
stopifnot(file.exists(pred_file))

tb_prediccion <- fread(pred_file, sep = "\t")

In [ ]:
setwd(PATH_THIS_EXPERIMENT)
kaggle <- "kaggle"
dir.create(kaggle, showWarnings= FALSE)

In [ ]:
res <- lapply(PARAM$cortes, function(corte) {
  armar_archivo_envios(
    pred         = tb_prediccion,
    corte        = corte,
    experimento  = TAG,  
    out_dir      = "kaggle",
    id_col       = "numero_de_cliente",
    prob_col     = "prob",
    sep_in       = "\t"
  )
})

In [ ]:
comp_name <- "dm-ey-f-2025-primera"  # p.ej.: "meic-dmeyf-2025"
cortes <- PARAM$cortes

# Loop para enviar cada uno
for (c in cortes) {
  enviar_archivo(
    pred         = tb_prediccion,  # o tu data.table con las probs
    corte        = c,
    experimento  = TAG,
    competition  = comp_name,    # slug exacto de Kaggle
    message_submit = sprintf("Exp=exp_lags1 | Corte=%d", c)
  )
}